In [2]:
import pandas as pd
import altair as alt

In [3]:
import pyarrow

In [4]:
import fastparquet

Let's take a look at the prices dataset first

In [5]:
prices_df = pd.read_parquet(
    'https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_prices.parquet',
    engine='fastparquet'
)
prices_df.describe()

,quote_date,shop_code,item_id_raw,region,price,item_id
count,4.836896e+07,4.836896e+07,4.836897e+07,4.836896e+07,4.836896e+07,4.836897e+07
mean,2.007762e+05,4.771270e+02,3.880409e+05,6.679112e+00,4.960396e+01,3.883983e+05
std,1.060226e+03,1.531775e+03,1.467557e+05,3.407499e+00,2.081768e+02,1.466723e+05
min,1.988020e+05,1.000000e+00,2.101010e+05,1.000000e+00,1.000000e-02,2.101010e+05
25%,1.998110e+05,3.900000e+01,2.129170e+05,3.000000e+00,1.490000e+00,2.129180e+05
50%,2.008050e+05,8.800000e+01,4.301280e+05,7.000000e+00,4.850000e+00,4.301320e+05
75%,2.017070e+05,8.020000e+02,5.104060e+05,9.000000e+00,1.999000e+01,5.104070e+05
max,2.025100e+05,2.007100e+04,6.404060e+05,1.300000e+01,4.400000e+04,6.404060e+05


We've got 48 million observations and a dates from 1982 to 2025.
</br></br></br></br>


We also need the items data to understand what each product is.

In [6]:
# Set pandas to display numbers without scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Now display the description again
prices_df.describe()

,quote_date,shop_code,item_id_raw,region,price,item_id
count,48368958.00,48368958.00,48368973.00,48368958.00,48368958.00,48368973.00
mean,200776.18,477.13,388040.87,6.68,49.60,388398.30
std,1060.23,1531.77,146755.65,3.41,208.18,146672.27
min,198802.00,1.00,210101.00,1.00,0.01,210101.00
25%,199811.00,39.00,212917.00,3.00,1.49,212918.00
50%,200805.00,88.00,430128.00,7.00,4.85,430132.00
75%,201707.00,802.00,510406.00,9.00,19.99,510407.00
max,202510.00,20071.00,640406.00,13.00,44000.00,640406.00


In [7]:
items_df = pd.read_parquet(
    'https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_item.parquet',
    engine='fastparquet'
)
items_df.head()

,item_id,description,date_quote_s,date_quote_e,n_obs
0,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
2,210105,LARGE WHOLEMEAL LOAF-UNSLICED,198802,200301,27161
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
4,210107,"BROWN LOAF,400G,SLICED-GRAN",198903,200401,29361


In [8]:
items_df[items_df['description'].str.contains('bread', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
5,210108,PITTA BREAD,200002,201001,14605
11,210114,CHILLED GARLIC BREAD,201002,202510,41672
13,210116,GLUTEN FREE BREAD LF 300-550G,202402,202510,2205
126,211211,FROZEN BREAD/BAT FISH 400-550G,201002,202510,40004
714,430417,BREAD BIN,199602,200101,8235


In [9]:
bread_prices = prices_df.query("item_id == 210106")
avg_bread_prices = bread_prices.groupby('quote_date').agg({'price': ['mean', 'median']}).reset_index()
    
avg_bread_prices.columns = ['date', 'Mean', 'Median']
avg_bread_prices

,date,Mean,Median
0,198802.00,0.48,0.47
1,198803.00,0.47,0.46
2,198804.00,0.47,0.47
3,198805.00,0.47,0.47
4,198806.00,0.48,0.48
...,...,...,...
446,202506.00,1.26,1.15
447,202507.00,1.25,1.15
448,202508.00,1.25,1.15
449,202509.00,1.27,1.17


In [10]:
bread_prices

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id
120117,199603.00,910.00,210106,6.00,0.67,,210106
120118,201708.00,2.00,210106,9.00,0.90,,210106
120119,202010.00,59.00,210106,3.00,0.70,,210106
120120,199208.00,14.00,210106,5.00,0.79,,210106
120121,201905.00,71.00,210106,3.00,0.70,,210106
...,...,...,...,...,...,...,...
187581,202405.00,941.00,210106,8.00,0.90,,210106
187582,199109.00,2.00,210106,12.00,0.40,,210106
187583,201607.00,802.00,210106,2.00,1.98,S,210106
187584,202410.00,801.00,210106,3.00,1.40,,210106


In [11]:
avg_bread_prices_melted = avg_bread_prices.melt(id_vars=['date'], 
                                                   var_name='price_type',
                                                   value_name='price')

# and we can also just format the date nicely, so altair treats them properly
avg_bread_prices_melted['date'] = pd.to_datetime(avg_bread_prices_melted['date'], format='%Y%m')

avg_bread_prices_melted

,date,price_type,price
0,1988-02-01,Mean,0.48
1,1988-03-01,Mean,0.47
2,1988-04-01,Mean,0.47
3,1988-05-01,Mean,0.47
4,1988-06-01,Mean,0.48
...,...,...,...
897,2025-06-01,Median,1.15
898,2025-07-01,Median,1.15
899,2025-08-01,Median,1.15
900,2025-09-01,Median,1.17


In [12]:
avg_bread_prices_melted.to_csv('avg_bread_prices_melted.csv', index=False)

In [20]:
byregion=bread_prices.groupby(['region', 'quote_date'])['price'].mean().reset_index()
byregion.columns = ['region', 'quote_date', 'avg_price']
byregion.to_csv('bread_prices_by_region.csv',index=False)
byregion

,region,quote_date,avg_price
0,2.00,198802.00,0.48
1,2.00,198803.00,0.51
2,2.00,198804.00,0.51
3,2.00,198805.00,0.51
4,2.00,198806.00,0.50
...,...,...,...
5407,13.00,202506.00,1.64
5408,13.00,202507.00,1.64
5409,13.00,202508.00,1.64
5410,13.00,202509.00,1.60
